In [ ]:
from IPython.display import HTML
HTML(open('../style.css').read())

# Jealous Couples


Three couples need to cross a river.  They have to follow the following rules: 
- The boat can only carry two persons.
- At least one person has to be on the boat on every crossing.
- The husbands know their wives well and would never leave them with another man, unless they are  present themselves.

We are going to formulate this problem as a *symbolic transition system*.
Then, we can solve the problem using the constraint solver `Z3`.

We assume the first couple is called `Anton` and `Doris`, the names of the second couple are
`Brian` and `Eliza`, while the last couple is called `Charly` and `Freya`.  Hence we
will use the following variables to encode the problem:
* `A` equals `1` if `Anton` is on the western shore,
* `B` equals `1` if `Brian` is on the western shore,
* `C` equals `1` if `Charly` is on the western shore,
* `D` equals `1` if `Doris` is on the western shore,
* `E` equals `1` if `Eliza` is on the western shore,
* `F` equals `1` if `Freya` is on the western shore,
* `S` equals `1` if the ship is on the westen shore.

In [ ]:
import z3

In [ ]:
A = z3.Int('A')
B = z3.Int('B')
C = z3.Int('C')
D = z3.Int('D')
E = z3.Int('E')
F = z3.Int('F')
S = z3.Int('S')

The function `start` returns a set of constrained that describe the initial state.

In [ ]:
def start(A, B, C, D, E, F, S):
    "your code here"

In [ ]:
start(A, B, C, D, E, F, S)

The function `goal` returns a set of constrained that describe the final state.

In [ ]:
def goal(A, B, C, D, E, F, S):
    "your code here"

In [ ]:
goal(A, B, C, D, E, F, S)

The function `invariant` returns a set of formulas that need to be true after every transition.

In [ ]:
def invariant(A, B, C, D, E, F, S):
    "your code here"

In [ ]:
invariant(A, B, C, D, E, F, S)

The function `transition` returns a set of formulas that describe the transitions.

In [ ]:
def transition(A, B, C, D, E, F, S, Ax, Bx, Cx, Dx, Ex, Fx, Sx):
    "your code here"

In [ ]:
def jealous_CSP(n):
    S = z3.Solver()
    As = [z3.Int(f'A{i}') for i in range(n+1)]
    Bs = [z3.Int(f'B{i}') for i in range(n+1)]
    Cs = [z3.Int(f'C{i}') for i in range(n+1)]
    Ds = [z3.Int(f'D{i}') for i in range(n+1)]
    Es = [z3.Int(f'E{i}') for i in range(n+1)]
    Fs = [z3.Int(f'F{i}') for i in range(n+1)]
    Ss = [z3.Int(f'S{i}') for i in range(n+1)]
    Cts  = start(As[0], Bs[0], Cs[0], Ds[0], Es[0], Fs[0], Ss[0])
    Cts |= goal( As[n], Bs[n], Cs[n], Ds[n], Es[n], Fs[n], Ss[n])
    for i in range(n):
        j = i+1
        Cts |= invariant( As[i], Bs[i], Cs[i], Ds[i], Es[i], Fs[i], Ss[i])
        Cts |= transition(As[i], Bs[i], Cs[i], Ds[i], Es[i], Fs[i], Ss[i],
                          As[j], Bs[j], Cs[j], Ds[j], Es[j], Fs[j], Ss[j])
        Cts.add(0 <= As[i])
        Cts.add(0 <= Bs[i])
        Cts.add(0 <= Cs[i])
        Cts.add(0 <= Ds[i])
        Cts.add(0 <= Es[i])
        Cts.add(0 <= Fs[i])
        Cts.add(0 <= Ss[i])
        Cts.add(As[i] <= 1) 
        Cts.add(Bs[i] <= 1)
        Cts.add(Cs[i] <= 1)
        Cts.add(Ds[i] <= 1)
        Cts.add(Es[i] <= 1)
        Cts.add(Fs[i] <= 1)
        Cts.add(Ss[i] <= 1)
    S.add(Cts)
    result = str(S.check())
    if result == 'sat':
        Model = S.model()
        Solution = (   { f'A{i}': Model[As[i]] for i in range(n+1) }
                     | { f'B{i}': Model[Bs[i]] for i in range(n+1) }
                     | { f'C{i}': Model[Cs[i]] for i in range(n+1) }
                     | { f'D{i}': Model[Ds[i]] for i in range(n+1) }
                     | { f'E{i}': Model[Es[i]] for i in range(n+1) }
                     | { f'F{i}': Model[Fs[i]] for i in range(n+1) }
                   )
        return { key: Solution[key].as_long() for key in Solution }
    else:
        return None

In [ ]:
def find_solution():
    n = 1
    while True:
        print(n)
        Solution = jealous_CSP(n)
        if Solution != None:
            return n, Solution
        n += 2

In [ ]:
%%time
n, Solution = find_solution()
n, Solution

## Auxiliary Code for Pretty Printing

The following code is used for printing the path that has been found.  We won't discuss the details of these functions.

In [ ]:
def show_solution(Solution, n):
    for i in range(n+1):
        A = Solution[f'A{i}']
        B = Solution[f'B{i}']
        C = Solution[f'C{i}']
        D = Solution[f'D{i}']
        E = Solution[f'E{i}']
        F = Solution[f'F{i}']
        print('👨'*A+'👨🏼'*B+'👨🏿'*C+'👩'*D+'👩🏼'*E+'👩🏿'*F + ' '*42 + \
              '👨'*(1-A)+'👨🏼'*(1-B)+'👨🏿'*(1-C)+'👩'*(1-D)+'👩🏼'*(1-E)+'👩🏿'*(1-F))
        if i % 2 == 0:
            AS = Solution[f'A{i}'] - Solution[f'A{i+1}']
            BS = Solution[f'B{i}'] - Solution[f'B{i+1}']
            CS = Solution[f'C{i}'] - Solution[f'C{i+1}']
            DS = Solution[f'D{i}'] - Solution[f'D{i+1}']
            ES = Solution[f'E{i}'] - Solution[f'E{i+1}']
            FS = Solution[f'F{i}'] - Solution[f'F{i+1}']
            print(' '*24+'>>>'+'👨'*AS+'👨🏼'*BS+'👨🏿'*CS+'👩'*DS+'👩🏼'*ES+'👩🏿'*FS+'>>>')
        elif i + 1 < n:
            AS = Solution[f'A{i+1}'] - Solution[f'A{i}']
            BS = Solution[f'B{i+1}'] - Solution[f'B{i}']
            CS = Solution[f'C{i+1}'] - Solution[f'C{i}']
            DS = Solution[f'D{i+1}'] - Solution[f'D{i}']
            ES = Solution[f'E{i+1}'] - Solution[f'E{i}']
            FS = Solution[f'F{i+1}'] - Solution[f'F{i}']
            print(' '*24+'<<<'+'👨'*AS+'👨🏼'*BS+'👨🏿'*CS+'👩'*DS+'👩🏼'*ES+'👩🏿'*FS+'<<<')

In [ ]:
show_solution(Solution, n)